In [1]:
!pip3 install -q transformers datasets # library and function import
from google.colab import drive
import os
import glob
from PIL import Image
import zipfile
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
drive.mount('/gdrive')
%cd /gdrive/My Drive/CLIP Project

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/.shortcut-targets-by-id/1Lbh9t5bjB1fmEf1UwwVEMp9R0aW_sIbd/CLIP Project


In [3]:
# check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type != 'cuda':
  raise SystemError('GPU device not found')

In [5]:
# Import unzipped images
# image_list = []
# for i, filename in enumerate(glob.glob('Train/resized_train/*.jpg')): #assuming gif
#   print(i, filename, type(filename))
#   if i==10:
#     break
  # im=Image.open(filename)
  # image_list.append(im)
print(len(glob.glob('Train/resized_train/*.jpg')))

0


In [10]:
# print(len(image_list))
captions = pd.read_csv('Train/caption_prediction_train.csv', header=None, sep='\t')

In [68]:
class ImageCaptioningDataset(Dataset):
  def __init__(self, train_captions, root_dir, transform=None, bert_model='distilbert-base-uncased', max_len=512):
    self.df = pd.read_csv(train_captions, header=None, sep='\t')
    self.root_dir = root_dir
    self.transform = transform
    self.tokenizer = AutoTokenizer.from_pretrained(bert_model)
    self.max_len = max_len

    self.images = self.df.iloc[1:,0]
    self.captions = self.df.iloc[1:,1]

  def __len__(self):
    return len(self.df)

  def generate_img_id(self, id):
    return "0"*len(str(6-id))+str(id)

  def __getitem__(self, idx):
    caption = self.captions[idx]
    # image = Image.open('Train/resized_train/ImageCLEFmedCaption_2022_train_'+self.generate_img_id(idx)+'.jpg')
    image_id = self.images[idx] + '.jpg'
    path_to_image = os.path.join(self.root_dir, image_id)
    image = Image.open(path_to_image).convert('RGB')
    
    if self.transform is not None:
      image = self.transform(image)

    tokenized_caption = self.tokenizer(caption, 
                                      padding='max_length',  # Pad to max_length
                                      truncation=True,  # Truncate to max_length
                                      max_length=self.max_len,  
                                      return_tensors='pt')['input_ids']
    
    return image, tokenized_caption

In [69]:
root_dir = 'Train/resized_train'
train_captions = 'Train/caption_prediction_train.csv'
bert_model = 'distilbert-base-uncased'

transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.PILToTensor()])

train_dataset = ImageCaptioningDataset(train_captions=train_captions,
                                       root_dir=root_dir,
                                       transform=transform,
                                       bert_model=bert_model)
train_loader = DataLoader(train_dataset, 
                          batch_size=64, 
                          num_workers=2, 
                          shuffle=True)

In [70]:
for batch_num, (image, caption) in enumerate(train_loader):
  if batch_num > 3:
    break
  print(f'batch number {batch_num} has {image.shape[0]} images and correspondingly {caption.shape[0]} tokenized captions')

FileNotFoundError: ignored